In [1]:
import numpy as np
import gym
import random

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\h5py\__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\scipy\fft\__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\scipy\sparse\sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAR

In [2]:
env = gym.make("CartPole-v1")

C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [3]:
env.action_space

Discrete(2)

In [4]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [6]:
import pygame
episodes = 15
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode {}: {}".format(episode, score))


Episode 0: 10.0
Episode 1: 16.0
Episode 2: 29.0
Episode 3: 17.0
Episode 4: 57.0
Episode 5: 33.0
Episode 6: 24.0
Episode 7: 12.0
Episode 8: 22.0
Episode 9: 11.0
Episode 10: 18.0
Episode 11: 33.0
Episode 12: 17.0
Episode 13: 23.0
Episode 14: 25.0


In [11]:
import tensorflow.keras.layers as layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [12]:
from tensorflow.keras.models import Sequential
def build_model(states, actions):
    model = Sequential()
    model.add(layers.Flatten(input_shape=(1,states)))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(actions, activation="linear"))
    return model

In [13]:
model = build_model(4, 2)

In [9]:
print(model)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
_________________________________________________________________


In [14]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, policy=policy, memory=memory, nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [15]:
dqn = build_agent(model, 2)
dqn.compile(Adam(learning_rate=1e-3), metrics=["mae"])

In [25]:
from callbacks import TrainEpisodeLogger, TrainIntervalLogger, TestLogger
callbacks = [TrainEpisodeLogger()]

In [26]:
# dqn.fit(env,nb_steps=100000, visualize=False, verbose=1)
dqn.fit(env,nb_steps=100000,visualize=False,verbose = 0, callbacks=callbacks)

Training for 100000 steps ...
   500/100000: episode: 1, duration: 7.014s, episode steps: 500, steps per second: 71, episode reward: 500.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.498 [0.000, 1.000], loss: 0.061578, mae: 59.982111, mean_q: 120.922146
The reward is higher than the best one, saving checkpoint weights
  1000/100000: episode: 2, duration: 7.080s, episode steps: 500, steps per second: 71, episode reward: 500.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.506 [0.000, 1.000], loss: 12.124689, mae: 59.141369, mean_q: 119.160362
The reward is lower than the best one, checkpoint weights not updated
  1500/100000: episode: 3, duration: 7.159s, episode steps: 500, steps per second: 70, episode reward: 500.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.498 [0.000, 1.000], loss: 7.326645, mae: 58.582142, mean_q: 118.301292
The reward is lower than the best one, checkpoint weights not updated
  1610/100000: episode: 4, duration: 1.608s, episode steps: 110,

In [27]:
scores = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

In [28]:
dqn.save_weights("./model/model.h5f", overwrite=True)

In [16]:
dqn.load_weights("./model/model.h5f")


In [ ]:
_ = dqn.test(env, nb_episodes=20, visualize=True)

Testing for 20 episodes ...


C:\Users\leff0\anaconda3\envs\ml\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
